In [35]:
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import pandas as pd
import cv2
import os
import numpy as np

In [36]:
IMAGE_WIDTH = 64
IMAGE_HEIGHT = 64
BATCH_SIZE = 32
NUM_EPOCHS = 10
NUM_CLASSES = 25

In [37]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH, 3))

In [38]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

In [39]:
model = Model(inputs=base_model.input, outputs=predictions)

In [40]:
for layer in base_model.layers:
    layer.trainable = True

In [48]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [50]:
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        directory='SML_Train',
        target_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
        batch_size=BATCH_SIZE,
        class_mode='categorical')

Found 16000 images belonging to 25 classes.


In [42]:
# Create the data generators for the training and testing images
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    'SML_Train',
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    subset='training')

Found 12812 images belonging to 25 classes.


In [43]:
validation_generator = train_datagen.flow_from_directory(
    'SML_Train',
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    subset='validation')

Found 3188 images belonging to 25 classes.


In [49]:
# Train the model
model.fit(train_generator,
          epochs=10,
          validation_data=validation_generator)

Epoch 1/10


2023-04-26 01:11:32.106255: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


401/401 [==============================] - ETA: 0s - loss: 0.2944 - accuracy: 0.9166

2023-04-26 01:12:06.859243: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


401/401 [==============================] - 37s 44ms/step - loss: 0.2944 - accuracy: 0.9166 - val_loss: 3.8195 - val_accuracy: 0.4991
Epoch 2/10
401/401 [==============================] - 17s 43ms/step - loss: 0.3132 - accuracy: 0.9109 - val_loss: 10.9969 - val_accuracy: 0.5116
Epoch 3/10
401/401 [==============================] - 17s 43ms/step - loss: 0.2156 - accuracy: 0.9371 - val_loss: 4.0981 - val_accuracy: 0.5047
Epoch 4/10
401/401 [==============================] - 17s 43ms/step - loss: 0.2533 - accuracy: 0.9271 - val_loss: 4.2732 - val_accuracy: 0.4172
Epoch 5/10
401/401 [==============================] - 17s 43ms/step - loss: 0.1864 - accuracy: 0.9422 - val_loss: 2.7586 - val_accuracy: 0.5339
Epoch 6/10
401/401 [==============================] - 17s 43ms/step - loss: 0.1746 - accuracy: 0.9482 - val_loss: 4.1799 - val_accuracy: 0.4150
Epoch 7/10
401/401 [==============================] - 17s 43ms/step - loss: 0.2142 - accuracy: 0.9326 - val_loss: 3.4692 - val_accuracy: 0.5016
Ep

In [51]:
model.fit(train_generator,
          epochs=10)

Epoch 1/10
  3/500 [..............................] - ETA: 19s - loss: 0.9363 - accuracy: 0.8021

2023-04-26 01:15:04.664920: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


500/500 [==============================] - 19s 37ms/step - loss: 0.4613 - accuracy: 0.8918
Epoch 2/10
500/500 [==============================] - 19s 37ms/step - loss: 0.1781 - accuracy: 0.9516
Epoch 3/10
500/500 [==============================] - 19s 37ms/step - loss: 0.1477 - accuracy: 0.9557
Epoch 4/10
500/500 [==============================] - 19s 37ms/step - loss: 0.1702 - accuracy: 0.9483
Epoch 5/10
500/500 [==============================] - 19s 38ms/step - loss: 0.1453 - accuracy: 0.9529
Epoch 6/10
500/500 [==============================] - 19s 38ms/step - loss: 0.1050 - accuracy: 0.9668
Epoch 7/10
500/500 [==============================] - 19s 38ms/step - loss: 0.1054 - accuracy: 0.9678
Epoch 8/10
500/500 [==============================] - 19s 38ms/step - loss: 0.1189 - accuracy: 0.9663
Epoch 9/10
500/500 [==============================] - 19s 38ms/step - loss: 0.0870 - accuracy: 0.9729
Epoch 10/10
500/500 [==============================] - 19s 38ms/step - loss: 0.0771 - accurac

In [52]:
# Create the testing data
test_files = os.listdir('SML_Test')
X_test = []
for file in test_files:
    img_path = os.path.join('SML_Test', file)
    img = cv2.imread(img_path)
    X_test.append(img)

# Convert the testing data to numpy arrays
X_test = np.array(X_test)

# Make predictions on the testing data
y_pred = model.predict(X_test)

# Save the predictions to a CSV file
test_df = pd.DataFrame({'id': test_files, 'category': np.argmax(y_pred, axis=1)})
test_df.to_csv('deepam-3.csv', index=False)

47/47 [==============================] - 1s 9ms/step
